

---

## 우리 데이터로 시각화 실험해보기
### 지역별 한부모 수급자 분포도 알아보기

### 환경설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 저장 시 폰트 깨지지 않도록 세팅

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [3]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [4]:
# 최신 버전 다운 시 이미지 저장 오류 발생! 5.3.1로 다운해야함
! pip install plotly==5.3.1

In [5]:
import folium

#### json 파일 불러오기

In [6]:
import requests
import json


r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [7]:
# json 파일 훑어보기
# 서울 행정구역 json raw파일(githubcontent) -> polygon data, 위도와 경도, 'code'로 구성되어 있음
# 강동구의 코드가 11250 => 확인 결과, 행정동 코드로 판명
seoul_geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.11879551821994, 37.557222485451305],
      [127.12146867175024, 37.55986003393365],
      [127.12435254630417, 37.56144246249796],
      [127.13593925898998, 37.56564793048277],
      [127.14930548011061, 37.56892250303897],
      [127.15511020940411, 37.57093642128295],
      [127.16683184366129, 37.57672487388627],
      [127.17038810813094, 37.576465605301046],
      [127.17607118428914, 37.57678573961056],
      [127.17905504160184, 37.57791388161732],
      [127.17747787800164, 37.57448983055031],
      [127.1781775408844, 37.571481967974336],
      [127.17995281860672, 37.569309661290504],
      [127.18122821955262, 37.56636089217979],
      [127.18169407550688, 37.56286338914073],
      [127.18408792

### 중위소득 총계 가구수 분포 구하기

In [8]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
medData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/한부모수급자_시각화용.xlsx', sheet_name = '중위소득별총계')

medData

,지역구,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수
0,종로구,중위소득총계,187,49,16
1,중구,중위소득총계,236,37,16
2,용산구,중위소득총계,350,58,28
3,성동구,중위소득총계,400,68,51
4,광진구,중위소득총계,739,140,72
5,동대문구,중위소득총계,661,115,57
6,중랑구,중위소득총계,1430,283,108
7,성북구,중위소득총계,944,161,76
8,강북구,중위소득총계,1128,216,86
9,도봉구,중위소득총계,1048,199,87


In [9]:
# 데이터 전처리 - 컬럼명 매칭
medData = medData.rename(columns={'지역구':'name'})
medData

,name,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수
0,종로구,중위소득총계,187,49,16
1,중구,중위소득총계,236,37,16
2,용산구,중위소득총계,350,58,28
3,성동구,중위소득총계,400,68,51
4,광진구,중위소득총계,739,140,72
5,동대문구,중위소득총계,661,115,57
6,중랑구,중위소득총계,1430,283,108
7,성북구,중위소득총계,944,161,76
8,강북구,중위소득총계,1128,216,86
9,도봉구,중위소득총계,1048,199,87


#### 중위소득 30% 이하 수급 가구수

In [10]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

medData30 = px.choropleth(medData, geojson=seoul_geo, locations='name', color='중위소득30이하_수급가구수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
medData30.update_geos(fitbounds="locations", visible=False)
medData30.update_layout(title_text='서울시 구별 중위소득 30% 이하 수급 가구수', title_font_size=20)

In [11]:
# 이미지 저장 위한 프로그램 다운로드
! pip install kaleido

In [12]:
# 프로그램 다운로드 2 -> 코랩 말고 터미널에서 실행해야 함
# conda install -c conda-forge python-kaleido

In [13]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
medData30.write_image('서울시 구별 중위소득 30% 이하 수급 가구수.png')

In [14]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
medData30.write_html("서울시 구별 중위소득 30% 이하 수급 가구수.html")

#### 중위소득 30~40 수급 가구수

In [15]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

medData3040 = px.choropleth(medData, geojson=seoul_geo, locations='name', color='중위소득3040_수급가구수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
medData3040.update_geos(fitbounds="locations", visible=False)
medData3040.update_layout(title_text='서울시 구별 중위소득 30~40% 수급 가구수', title_font_size=20)

In [16]:
# 이미지 저장 위한 프로그램 다운로드
! pip install kaleido

In [17]:
# 프로그램 다운로드 2 -> 코랩 말고 터미널에서 실행해야 함
# conda install -c conda-forge python-kaleido

In [18]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
medData3040.write_image('서울시 구별 중위소득 30~40% 수급 가구수.png')

In [19]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
medData3040.write_html("서울시 구별 중위소득 30~40% 수급 가구수.html")

#### 중위소득 40~50 수급 가구수

In [20]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

medData4050 = px.choropleth(medData, geojson=seoul_geo, locations='name', color='중위소득4050_수급가구수',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
medData4050.update_geos(fitbounds="locations", visible=False)
medData4050.update_layout(title_text='서울시 구별 중위소득 40~50% 수급 가구수', title_font_size=20)

In [21]:
# 이미지 저장 위한 프로그램 다운로드
! pip install kaleido

In [22]:
# 프로그램 다운로드 2 -> 코랩 말고 터미널에서 실행해야 함
# conda install -c conda-forge python-kaleido

In [23]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
medData4050.write_image('서울시 구별 중위소득 40~50% 수급 가구수.png')

In [24]:
# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
medData4050.write_html("서울시 구별 중위소득 40~50% 수급 가구수.html")



---

### 전체 수혜자 중, 소득 수준 비율

In [25]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
medratioData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/test2.xlsx', sheet_name = '중위소득별소득수준비')

medratioData

,지역구,가족유형,중위소득30이하_수급가구비,중위소득3040_수급가구비,중위소득4050_수급가구비
0,종로구,중위소득총계,0.007577,0.001985,0.000648
1,중구,중위소득총계,0.009563,0.001499,0.000648
2,용산구,중위소득총계,0.014182,0.002350,0.001135
3,성동구,중위소득총계,0.016208,0.002755,0.002067
4,광진구,중위소득총계,0.029944,0.005673,0.002917
5,동대문구,중위소득총계,0.026784,0.004660,0.002310
6,중랑구,중위소득총계,0.057944,0.011467,0.004376
7,성북구,중위소득총계,0.038251,0.006524,0.003080
8,강북구,중위소득총계,0.045707,0.008752,0.003485
9,도봉구,중위소득총계,0.042465,0.008064,0.003525


In [26]:
# 데이터 전처리 - 컬럼명 매칭
medratioData = medratioData.rename(columns={'지역구':'name'})
medratioData

,name,가족유형,중위소득30이하_수급가구비,중위소득3040_수급가구비,중위소득4050_수급가구비
0,종로구,중위소득총계,0.007577,0.001985,0.000648
1,중구,중위소득총계,0.009563,0.001499,0.000648
2,용산구,중위소득총계,0.014182,0.002350,0.001135
3,성동구,중위소득총계,0.016208,0.002755,0.002067
4,광진구,중위소득총계,0.029944,0.005673,0.002917
5,동대문구,중위소득총계,0.026784,0.004660,0.002310
6,중랑구,중위소득총계,0.057944,0.011467,0.004376
7,성북구,중위소득총계,0.038251,0.006524,0.003080
8,강북구,중위소득총계,0.045707,0.008752,0.003485
9,도봉구,중위소득총계,0.042465,0.008064,0.003525


#### 중위소득 30% 이하 수급 가구비

In [27]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

medratio30 = px.choropleth(medratioData, geojson=seoul_geo, locations='name', color='중위소득30이하_수급가구비',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
medratio30.update_geos(fitbounds="locations", visible=False)
medratio30.update_layout(title_text='서울시 구별 중위소득 30% 이하 수급 가구 비율', title_font_size=20)

In [28]:
# 이미지 저장 위한 프로그램 다운로드
! pip install kaleido

In [29]:
# 프로그램 다운로드 2 -> 코랩 말고 터미널에서 실행해야 함
# conda install -c conda-forge python-kaleido

In [30]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
medratio30.write_image('서울시 구별 중위소득 30% 이하 수급 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
medratio30.write_html("서울시 구별 중위소득 30% 이하 수급 가구 비율.html")

#### 중위소득 30~40% 수급 가구 비

In [31]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

medratio3040 = px.choropleth(medratioData, geojson=seoul_geo, locations='name', color='중위소득3040_수급가구비',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
medratio3040.update_geos(fitbounds="locations", visible=False)
medratio3040.update_layout(title_text='서울시 구별 중위소득 30~40% 수급 가구 비율', title_font_size=20)

In [32]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
medratio3040.write_image('서울시 구별 중위소득 30~40% 수급 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
medratio3040.write_html("서울시 구별 중위소득 30~40% 수급 가구 비율.html")

#### 중위소득 40~50% 수급 가구 비

In [33]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

medratio4050 = px.choropleth(medratioData, geojson=seoul_geo, locations='name', color='중위소득4050_수급가구비',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
medratio4050.update_geos(fitbounds="locations", visible=False)
medratio4050.update_layout(title_text='서울시 구별 중위소득 40~50% 수급 가구 비율', title_font_size=20)

In [34]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
medratio4050.write_image('서울시 구별 중위소득 40~50% 수급 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
medratio4050.write_html("서울시 구별 중위소득 40~50% 수급 가구 비율.html")



---

## 유형 비율 별, 소득 수준 비율

### 중위소득총계

In [35]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
charIncData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/test2.xlsx', sheet_name = '중위소득총계별소득수준비')

charIncData

,지역구,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,성동구,중위소득총계,400,68,51,0.770713,0.131021,0.098266
1,영등포구,중위소득총계,407,71,41,0.784200,0.136802,0.078998
2,송파구,중위소득총계,975,204,99,0.762911,0.159624,0.077465
3,강서구,중위소득총계,1432,314,144,0.757672,0.166138,0.076190
4,광진구,중위소득총계,739,140,72,0.777077,0.147213,0.075710
5,은평구,중위소득총계,1222,236,113,0.777849,0.150223,0.071929
6,서대문구,중위소득총계,595,103,53,0.792277,0.137150,0.070573
7,동대문구,중위소득총계,661,115,57,0.793517,0.138055,0.068427
8,동작구,중위소득총계,493,106,43,0.767913,0.165109,0.066978
9,관악구,중위소득총계,891,168,75,0.785714,0.148148,0.066138


In [36]:
# 데이터 전처리 - 컬럼명 매칭
charIncData = charIncData.rename(columns={'지역구':'name'})
charIncData

,name,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,성동구,중위소득총계,400,68,51,0.770713,0.131021,0.098266
1,영등포구,중위소득총계,407,71,41,0.784200,0.136802,0.078998
2,송파구,중위소득총계,975,204,99,0.762911,0.159624,0.077465
3,강서구,중위소득총계,1432,314,144,0.757672,0.166138,0.076190
4,광진구,중위소득총계,739,140,72,0.777077,0.147213,0.075710
5,은평구,중위소득총계,1222,236,113,0.777849,0.150223,0.071929
6,서대문구,중위소득총계,595,103,53,0.792277,0.137150,0.070573
7,동대문구,중위소득총계,661,115,57,0.793517,0.138055,0.068427
8,동작구,중위소득총계,493,106,43,0.767913,0.165109,0.066978
9,관악구,중위소득총계,891,168,75,0.785714,0.148148,0.066138


#### 중위소득 30% 이하 수급 가구비

In [37]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

charInc30 = px.choropleth(charIncData, geojson=seoul_geo, locations='name', color='30이하비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
charInc30.update_geos(fitbounds="locations", visible=False)
charInc30.update_layout(title_text='서울시 지역 내 중위소득 30% 이하 수급 가구 비율', title_font_size=20)

In [38]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
charInc30.write_image('서울시 지역 내 중위소득 30% 이하 수급 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
charInc30.write_html("서울시 지역 내 중위소득 30% 이하 수급 가구 비율.html")

#### 중위소득 30~40% 수급 가구 비

In [39]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

charInc3040 = px.choropleth(charIncData, geojson=seoul_geo, locations='name', color='3040비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
charInc3040.update_geos(fitbounds="locations", visible=False)
charInc3040.update_layout(title_text='서울시 지역 내 중위소득 30~40% 수급 가구 비율', title_font_size=20)

In [40]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
charInc3040.write_image('서울시 지역 내 중위소득 30~40% 수급 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
charInc3040.write_html("서울시 지역 내 중위소득 30~40% 수급 가구 비율.html")

#### 중위소득 40~50% 수급 가구 비

In [41]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

charInc4050 = px.choropleth(charIncData, geojson=seoul_geo, locations='name', color='4050비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
charInc4050.update_geos(fitbounds="locations", visible=False)
charInc4050.update_layout(title_text='서울시 지역 내 중위소득 40~50% 수급 가구 비율', title_font_size=20)

In [42]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
charInc4050.write_image('서울시 지역 내 중위소득 40~50% 수급 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
charInc4050.write_html("서울시 지역 내 중위소득 40~50% 수급 가구 비율.html")

### 부자가족별소득수준비

In [43]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
dadIncData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/test2.xlsx', sheet_name = '부자가족별소득수준비')

dadIncData

,지역구,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,종로구,부자가족,44,10,5,0.745763,0.169492,0.084746
1,도봉구,부자가족,195,32,21,0.786290,0.129032,0.084677
2,용산구,부자가족,49,8,5,0.790323,0.129032,0.080645
3,은평구,부자가족,215,34,21,0.796296,0.125926,0.077778
4,송파구,부자가족,163,42,16,0.737557,0.190045,0.072398
5,성동구,부자가족,83,20,8,0.747748,0.180180,0.072072
6,강북구,부자가족,207,38,19,0.784091,0.143939,0.071970
7,관악구,부자가족,177,37,16,0.769565,0.160870,0.069565
8,강서구,부자가족,272,49,24,0.788406,0.142029,0.069565
9,영등포구,부자가족,80,17,7,0.769231,0.163462,0.067308


In [44]:
# 데이터 전처리 - 컬럼명 매칭
dadIncData = dadIncData.rename(columns={'지역구':'name'})
dadIncData

,name,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,종로구,부자가족,44,10,5,0.745763,0.169492,0.084746
1,도봉구,부자가족,195,32,21,0.786290,0.129032,0.084677
2,용산구,부자가족,49,8,5,0.790323,0.129032,0.080645
3,은평구,부자가족,215,34,21,0.796296,0.125926,0.077778
4,송파구,부자가족,163,42,16,0.737557,0.190045,0.072398
5,성동구,부자가족,83,20,8,0.747748,0.180180,0.072072
6,강북구,부자가족,207,38,19,0.784091,0.143939,0.071970
7,관악구,부자가족,177,37,16,0.769565,0.160870,0.069565
8,강서구,부자가족,272,49,24,0.788406,0.142029,0.069565
9,영등포구,부자가족,80,17,7,0.769231,0.163462,0.067308


#### 중위소득 30% 이하 수급 가구비

In [45]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

dadInc30 = px.choropleth(dadIncData, geojson=seoul_geo, locations='name', color='30이하비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
dadInc30.update_geos(fitbounds="locations", visible=False)
dadInc30.update_layout(title_text='서울시 지역 내 중위소득 30% 이하 수급 부자 가족 가구 비율', title_font_size=20)

In [46]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
dadInc30.write_image('서울시 지역 내 중위소득 30% 이하 수급 부자 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
dadInc30.write_html("서울시 지역 내 중위소득 30% 이하 수급 부자 가족 가구 비율.html")

#### 중위소득 30~40% 수급 가구 비

In [47]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

dadInc3040 = px.choropleth(dadIncData, geojson=seoul_geo, locations='name', color='3040비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
dadInc3040.update_geos(fitbounds="locations", visible=False)
dadInc3040.update_layout(title_text='서울시 지역 내 중위소득 30~40% 수급 부자 가족 가구 비율', title_font_size=20)

In [48]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
dadInc3040.write_image('서울시 지역 내 중위소득 30~40% 수급 부자 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
dadInc3040.write_html("서울시 지역 내 중위소득 30~40% 수급 부자 가족 가구 비율.html")

#### 중위소득 40~50% 수급 가구 비

In [49]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

dadInc4050 = px.choropleth(dadIncData, geojson=seoul_geo, locations='name', color='4050비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
dadInc4050.update_geos(fitbounds="locations", visible=False)
dadInc4050.update_layout(title_text='서울시 지역 내 중위소득 40~50% 수급 부자 가족 가구 비율', title_font_size=20)

In [50]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
dadInc4050.write_image('서울시 지역 내 중위소득 40~50% 수급 부자 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
dadInc4050.write_html("서울시 지역 내 중위소득 40~50% 수급 부자 가족 가구 비율.html")

### 모자가족별소득수준비

In [51]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
momIncData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/test2.xlsx', sheet_name = '모자가족별소득수준비')

momIncData

,지역구,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,성동구,모자가족,309,47,34,0.792308,0.120513,0.087179
1,영등포구,모자가족,318,54,34,0.783251,0.133005,0.083744
2,광진구,모자가족,591,112,62,0.772549,0.146405,0.081046
3,송파구,모자가족,794,161,82,0.765670,0.155256,0.079074
4,강서구,모자가족,1125,259,117,0.749500,0.172552,0.077948
5,서대문구,모자가족,468,86,45,0.781302,0.143573,0.075125
6,동대문구,모자가족,503,84,46,0.794629,0.132701,0.072670
7,은평구,모자가족,986,200,92,0.771518,0.156495,0.071987
8,동작구,모자가족,395,88,37,0.759615,0.169231,0.071154
9,성북구,모자가족,763,132,68,0.792316,0.137072,0.070613


In [52]:
# 데이터 전처리 - 컬럼명 매칭
momIncData = momIncData.rename(columns={'지역구':'name'})
momIncData

,name,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,성동구,모자가족,309,47,34,0.792308,0.120513,0.087179
1,영등포구,모자가족,318,54,34,0.783251,0.133005,0.083744
2,광진구,모자가족,591,112,62,0.772549,0.146405,0.081046
3,송파구,모자가족,794,161,82,0.765670,0.155256,0.079074
4,강서구,모자가족,1125,259,117,0.749500,0.172552,0.077948
5,서대문구,모자가족,468,86,45,0.781302,0.143573,0.075125
6,동대문구,모자가족,503,84,46,0.794629,0.132701,0.072670
7,은평구,모자가족,986,200,92,0.771518,0.156495,0.071987
8,동작구,모자가족,395,88,37,0.759615,0.169231,0.071154
9,성북구,모자가족,763,132,68,0.792316,0.137072,0.070613


#### 중위소득 30% 이하 수급 가구비

In [53]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

momInc30 = px.choropleth(momIncData, geojson=seoul_geo, locations='name', color='30이하비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
momInc30.update_geos(fitbounds="locations", visible=False)
momInc30.update_layout(title_text='서울시 지역 내 중위소득 30% 이하 수급 모자 가족 가구 비율', title_font_size=20)

In [54]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
momInc30.write_image('서울시 지역 내 중위소득 30% 이하 수급 모자 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
momInc30.write_html("서울시 지역 내 중위소득 30% 이하 수급 모자 가족 가구 비율.html")

#### 중위소득 30~40% 수급 가구 비

In [55]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

momInc3040 = px.choropleth(momIncData, geojson=seoul_geo, locations='name', color='3040비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
momInc3040.update_geos(fitbounds="locations", visible=False)
momInc3040.update_layout(title_text='서울시 지역 내 중위소득 30~40% 수급 모자 가족 가구 비율', title_font_size=20)

In [56]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
momInc3040.write_image('서울시 지역 내 중위소득 30~40% 수급 모자 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
momInc3040.write_html("서울시 지역 내 중위소득 30~40% 수급 모자 가족 가구 비율.html")

#### 중위소득 40~50% 수급 가구 비

In [57]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

momInc4050 = px.choropleth(momIncData, geojson=seoul_geo, locations='name', color='4050비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
momInc4050.update_geos(fitbounds="locations", visible=False)
momInc4050.update_layout(title_text='서울시 지역 내 중위소득 40~50% 수급 모자 가족 가구 비율', title_font_size=20)

In [58]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
momInc4050.write_image('서울시 지역 내 중위소득 40~50% 수급 모자 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
momInc4050.write_html("서울시 지역 내 중위소득 40~50% 수급 모자 가족 가구 비율.html")

### 조손가족별소득수준비

In [59]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
grdIncData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/test2.xlsx', sheet_name = '조손가족별소득수준비')

grdIncData

,지역구,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,구로구,조손가족,0,0,1,0.000000,0.000000,1.000000
1,성동구,조손가족,6,0,8,0.428571,0.000000,0.571429
2,강동구,조손가족,2,0,1,0.666667,0.000000,0.333333
3,동작구,조손가족,8,0,1,0.888889,0.000000,0.111111
4,강서구,조손가족,21,2,1,0.875000,0.083333,0.041667
5,양천구,조손가족,1,1,0,0.500000,0.500000,0.000000
6,서대문구,조손가족,7,1,0,0.875000,0.125000,0.000000
7,성북구,조손가족,10,1,0,0.909091,0.090909,0.000000
8,중랑구,조손가족,11,1,0,0.916667,0.083333,0.000000
9,중구,조손가족,2,0,0,1.000000,0.000000,0.000000


In [60]:
# 데이터 전처리 - 컬럼명 매칭
grdIncData = grdIncData.rename(columns={'지역구':'name'})
grdIncData

,name,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,구로구,조손가족,0,0,1,0.000000,0.000000,1.000000
1,성동구,조손가족,6,0,8,0.428571,0.000000,0.571429
2,강동구,조손가족,2,0,1,0.666667,0.000000,0.333333
3,동작구,조손가족,8,0,1,0.888889,0.000000,0.111111
4,강서구,조손가족,21,2,1,0.875000,0.083333,0.041667
5,양천구,조손가족,1,1,0,0.500000,0.500000,0.000000
6,서대문구,조손가족,7,1,0,0.875000,0.125000,0.000000
7,성북구,조손가족,10,1,0,0.909091,0.090909,0.000000
8,중랑구,조손가족,11,1,0,0.916667,0.083333,0.000000
9,중구,조손가족,2,0,0,1.000000,0.000000,0.000000


#### 중위소득 30% 이하 수급 가구비

In [61]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

grdInc30 = px.choropleth(grdIncData, geojson=seoul_geo, locations='name', color='30이하비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
grdInc30.update_geos(fitbounds="locations", visible=False)
grdInc30.update_layout(title_text='서울시 지역 내 중위소득 30% 이하 수급 조손 가족 가구 비율', title_font_size=20)

In [62]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
grdInc30.write_image('서울시 지역 내 중위소득 30% 이하 수급 조손 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
grdInc30.write_html("서울시 지역 내 중위소득 30% 이하 수급 조손 가족 가구 비율.html")

#### 중위소득 30~40% 수급 가구 비

In [63]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

grdInc3040 = px.choropleth(grdIncData, geojson=seoul_geo, locations='name', color='3040비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
grdInc3040.update_geos(fitbounds="locations", visible=False)
grdInc3040.update_layout(title_text='서울시 지역 내 중위소득 30~40% 수급 조손 가족 가구 비율', title_font_size=20)

In [64]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
grdInc3040.write_image('서울시 지역 내 중위소득 30~40% 수급 조손 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
grdInc3040.write_html("서울시 지역 내 중위소득 30~40% 수급 조손 가족 가구 비율.html")

#### 중위소득 40~50% 수급 가구 비

In [65]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

grdInc4050 = px.choropleth(grdIncData, geojson=seoul_geo, locations='name', color='4050비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
grdInc4050.update_geos(fitbounds="locations", visible=False)
grdInc4050.update_layout(title_text='서울시 지역 내 중위소득 40~50% 수급 조손 가족 가구 비율', title_font_size=20)

In [66]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
grdInc4050.write_image('서울시 지역 내 중위소득 40~50% 수급 조손 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
grdInc4050.write_html("서울시 지역 내 중위소득 40~50% 수급 조손 가족 가구 비율.html")

### 청소년한부모별소득수준비


In [67]:
import pandas as pd

# str 값으로 가져올 경우 (시군구명) str 변환 불필요
# 총합 시설 알아보기
kidIncData = pd.read_excel('/content/drive/MyDrive/2023 데청캠/프로젝트/test2.xlsx', sheet_name = '청소년한부모별소득수준비')

kidIncData

,지역구,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,종로구,청소년한부모,3,1,0,0.750000,0.250000,0.000000
1,중구,청소년한부모,0,0,0,0.000000,0.000000,0.000000
2,성동구,청소년한부모,2,1,1,0.500000,0.250000,0.250000
3,관악구,청소년한부모,6,3,1,0.600000,0.300000,0.100000
4,강서구,청소년한부모,14,4,2,0.700000,0.200000,0.100000
5,동대문구,청소년한부모,3,1,0,0.750000,0.250000,0.000000
6,노원구,청소년한부모,10,2,1,0.769231,0.153846,0.076923
7,강북구,청소년한부모,7,1,1,0.777778,0.111111,0.111111
8,은평구,청소년한부모,9,2,0,0.818182,0.181818,0.000000
9,용산구,청소년한부모,5,1,0,0.833333,0.166667,0.000000


In [68]:
# 데이터 전처리 - 컬럼명 매칭
kidIncData = kidIncData.rename(columns={'지역구':'name'})
kidIncData

,name,가족유형,중위소득30이하_수급가구수,중위소득3040_수급가구수,중위소득4050_수급가구수,30이하비율,3040비율,4050비율
0,종로구,청소년한부모,3,1,0,0.750000,0.250000,0.000000
1,중구,청소년한부모,0,0,0,0.000000,0.000000,0.000000
2,성동구,청소년한부모,2,1,1,0.500000,0.250000,0.250000
3,관악구,청소년한부모,6,3,1,0.600000,0.300000,0.100000
4,강서구,청소년한부모,14,4,2,0.700000,0.200000,0.100000
5,동대문구,청소년한부모,3,1,0,0.750000,0.250000,0.000000
6,노원구,청소년한부모,10,2,1,0.769231,0.153846,0.076923
7,강북구,청소년한부모,7,1,1,0.777778,0.111111,0.111111
8,은평구,청소년한부모,9,2,0,0.818182,0.181818,0.000000
9,용산구,청소년한부모,5,1,0,0.833333,0.166667,0.000000


#### 중위소득 30% 이하 수급 가구 비

In [69]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

kidInc30 = px.choropleth(kidIncData, geojson=seoul_geo, locations='name', color='30이하비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
kidInc30.update_geos(fitbounds="locations", visible=False)
kidInc30.update_layout(title_text='서울시 지역 내 중위소득 30% 이하 수급 청소년한부모 가족 가구 비율', title_font_size=20)

In [70]:
# 이미지 저장 위한 프로그램 다운로드
! pip install kaleido

In [71]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
kidInc30.write_image('서울시 지역 내 중위소득 30% 이하 수급 청소년한부모 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
kidInc30.write_html("서울시 지역 내 중위소득 30% 이하 수급 청소년한부모 가족 가구 비율.html")

#### 중위소득 30~40% 수급 가구 비

In [72]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

kidInc3040 = px.choropleth(kidIncData, geojson=seoul_geo, locations='name', color='3040비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
kidInc3040.update_geos(fitbounds="locations", visible=False)
kidInc3040.update_layout(title_text='서울시 지역 내 중위소득 30~40% 수급 청소년한부모 가족 가구 비율', title_font_size=20)

In [73]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
kidInc3040.write_image('서울시 지역 내 중위소득 30~40% 수급 청소년한부모 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
kidInc3040.write_html("서울시 지역 내 중위소득 30~40% 수급 청소년한부모 가족 가구 비율.html")

#### 중위소득 40~50% 수급 가구 비

In [74]:
# folium 패키지로 값 기반 map 생성

import plotly.express as px

# locations : dataframe, geojson 모두 같은 변수명인 값 (code/name)
# color : 시각화 할 때 색으로 드러나는 기준이 되는 값

kidInc4050 = px.choropleth(kidIncData, geojson=seoul_geo, locations='name', color='4050비율',
                                color_continuous_scale='Blues',
                                featureidkey='properties.name')
kidInc4050.update_geos(fitbounds="locations", visible=False)
kidInc4050.update_layout(title_text='서울시 지역 내 중위소득 40~50% 수급 청소년한부모 가족 가구 비율', title_font_size=20)

In [75]:
# 시각화 파일을 png로 저장
import os

os.chdir('/content/drive/MyDrive/2023 데청캠/프로젝트/medDatas')
kidInc4050.write_image('서울시 지역 내 중위소득 40~50% 수급 청소년한부모 가족 가구 비율.png')

# 시각화 파일을 html로 저장 : 클릭해서 내용을 볼 수 있다는 점에서 위 아래 모두 하는 게 좋을 듯
kidInc4050.write_html("서울시 지역 내 중위소득 40~50% 수급 청소년한부모 가족 가구 비율.html")